# Using Wallaroo Shadow Deployment in Computer Vision - Part 2

Now that we have learned a little bit about how both object detectors perform, what if we could have the best of both worlds.

In this tutorial we will once again configure the pipeline to use a shadow deployment where the control is the production mobilenet object detector.  We will configure a challenger resnet50 object detector.

Both object detectors will once again perform inference on each frame in the video and the objects they detect, <b>but only the object detector with the highest average confidence score will be used.</b>

Our goal is to increase our inference success rate from 85% to 95% by choosing the object detector with the highest avareage confidence score in each frame.

After you execute this notebook and download the video, you might want to set the playback speed to 0.25 or 0.50 to see the results at a speed we humans can process.


In [1]:
!pip install imutils
!pip install pytz
!pip install more-itertools

import torch
import pickle
import wallaroo
import os
import numpy as np
import json
import requests
import time
import imutils
from CVDemoUtils import CVDemo


/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialize some Vars

Initialize the COCO Classes, meaning the classificaitons found on the images and the default width and height all images are resized to

In [2]:
wl = wallaroo.Client()

In [4]:
ws = wl.list_workspaces()
for w in ws:
    if w.name() == 'computer-vision':
        wl.set_current_workspace(w)

In [5]:
control =  wl.upload_model('mobilenet', 'models/mobilenet.pt.onnx')

In [6]:
challenger_list  = [ 
    wl.upload_model('resnet50', 'models/frcnn-resnet.pt.onnx')
]

In [7]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(2).memory("8Gi").build()

In [8]:
pipeline = wl.build_pipeline("shadow-pp")
pipeline.add_shadow_deploy(control, challenger_list)
pipeline.deploy(deployment_config = deployment_config)

Waiting for deployment - this will take up to 45s ..... ok


name,shadow-pp
created,2022-11-08 20:04:56.582490+00:00
last_updated,2022-11-09 15:56:10.916084+00:00
deployed,True
tags,
steps,resnet50


In [9]:
time.sleep(5) # needed to allow the pipeline to settle in.
url = pipeline._deployment._url()
print(url)

http://engine-lb.shadow-pp-895:29502/pipelines/shadow-pp


In [10]:

from CVDemoUtils import CVDemo

# set the device we will be using to run the model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# The set of COCO classifications
CLASSES = pickle.loads(open("models/coco_classes.pickle", "rb").read())

# Unique colors for each identified COCO class
#COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))
COLORS = [CVDemo.BLUE, CVDemo.RED]

# The size the image will be resized to
width = 640
height = 480

# Only objects that have a confidence > confidence_target will be displayed on the image
confidence_target = 0.75

cvDemo = CVDemo(CLASSES,COLORS, DEVICE)

### Detect and Classify Objects in Video using Wallaroo Shadow Deployment

TBD

In [11]:
cvDemo = CVDemo(CLASSES,COLORS, DEVICE)

# Only objects that have a confidence > confidence_target will be displayed on the image
input_video = "videos/amazon-fresh-go.mp4"
input_video = "videos/ww2-warbirds-in-formation.mp4"
output_video = "videos/ww2-warbirds-in-formation-inferenced.mp4"

cur_ws = wl.get_current_workspace()
config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : 25, # Frames per second
    'pipeline' : pipeline,
    'control-model' : control,
    'challenger-model-list' : challenger_list,
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    'max-frames' : 1000, # the # of frames to capture in the output video
    'skip-frames' : 100, # the # of frames to capture in the output video
    'confidence-target' : 0.90, # only display bounding boxes with confidence > provided #
    'model-cnt': 2,
    'inference': 'WALLAROO_SDK',
    'color': CVDemo.BLUE
}
cvDemo.DEBUG = False
cvDemo.useBestObjectDetectorInVideoUsingShadowDeployment(config)
print("We are done.")





11-09-2022 15:56:21.861089 Start Time:10:56:21 AM
11-09-2022 15:56:21.861263 Skipping [100] frames
11-09-2022 15:56:21.861278 Captureing up to [1000] frames
11-09-2022 15:56:21.871960 Video Properties
11-09-2022 15:56:21.872033    format:0.0
11-09-2022 15:56:21.872044    fourcc:828601953.0
11-09-2022 15:56:21.872052    mode:0.0
11-09-2022 15:56:21.872060    buffer:0.0
11-09-2022 15:56:21.872067    width:1280.0
11-09-2022 15:56:21.872075    height:720.0
11-09-2022 15:56:21.872082    fps:25.0
11-09-2022 15:56:21.872089    frame count:12721.0
11-09-2022 15:56:25.388767 best model=mobilenet
11-09-2022 15:56:25.393721 Frame:1 Read: 0.0010 Inf: 3.3042 Onnx: 0.0000 Draw: 0.0047 Total: 3.3571
11-09-2022 15:56:29.182804 best model=mobilenet
11-09-2022 15:56:29.187375 Frame:2 Read: 0.0014 Inf: 3.7453 Onnx: 0.0000 Draw: 0.0044 Total: 3.7936
11-09-2022 15:56:32.453588 best model=mobilenet
11-09-2022 15:56:32.458737 Frame:3 Read: 0.0011 Inf: 3.2230 Onnx: 0.0000 Draw: 0.0047 Total: 3.2713
11-09-2022

In [11]:
#for d in wl.list_deployments():
#    d.undeploy()
    

### Conclusion

Notice the difference in the control confidence and the challenger confidence.  Clearly we can see in this example the Control resnet50 model is performing better than the mobilenet.  This is likely due to the fact that frcnn resnet50 model is a 2 stage object detector vs the mobilenet is a single stage detector.

In the next example we will take this to another level and see what happens when we use shadow deployment in a video in order to achieve the best of both worlds.  Highest accuracy with the best performance.